# Neural network implementation

In [1]:
import numpy as np
import os

from fault_tolerant_ml.data.mnist import MNist
import fault_tolerant_ml.activations.activation as F

%reload_ext autoreload
%autoreload 2

## Read in data

In [2]:
data_dir = "../data"
filepaths = {
    "train": {
        "images": os.path.join(data_dir, "train-images-idx3-ubyte.gz"), "labels": os.path.join(data_dir, "train-labels-idx1-ubyte.gz")
    },
    "test": {
        "images": os.path.join(data_dir, "t10k-images-idx3-ubyte.gz"), "labels": os.path.join(data_dir, "t10k-labels-idx1-ubyte.gz")
    }
}
mnist = MNist(filepaths)

In [3]:
mnist

<MNist X_train=(60000, 784), y_train=(60000, 10), X_test=(10000, 784), y_test=(10000, 10)>

In [4]:
n_features, n_classes = mnist.X_train.shape[1], mnist.y_train.shape[1]
print(f"n_features={n_features}, n_classes={n_classes}")

n_features=784, n_classes=10


In [5]:
# class NeuralNet(nn.Model):
    
#     def __init__(self):
        
#         # MLP - 1 input layer, 1 hidden layer, 1 output layer
#         # self.fc1 = nn.Layer(n_inputs=784, n_outputs=128)
#         # self.fc2 = nn.Layer(n_inputs=128, n_outputs=10)
#         self.layers = []
#         # self.act_fn = F.Sigmoid()
        
#     def add(self, layer):
#         """Add layer to model
#         """
#         self.layers.append(layer)
#         return self
    
#     def forward(self, x):
        
#         # z1 = self.fc1(x)
#         # a1 = self.act_fn(z1)
#         # a1, z1 = self.fc1(x)
#         # z2 = self.fc2(a1)
#         # y_pred = self.act_fn(z2)
#         # y_pred, z2 = self.fc2(a1)
        
#         a = x
#         for layer in self.layers:
#             print(layer)
#             layer(a)
#             a = layer.y
        
#         y_pred = a
#         return y_pred
    
#     def backward():
#         pass
    
# #     def backward(self, x, y, a_n):
        
# #         y_pred, z2, a1, z1 = a_n
# #         # Output layer error
# #         delta2 = (y_pred - y)# * self.act_fn.grad(z2)
# #         # Gradient of cost function
# #         dw2 = np.dot(a1.T, delta2)
# #         # Backpropagate the error through the network
# #         delta1 = np.dot(delta2, self.fc2.W.T) * self.act_fn.grad(z1)
# #         # Calculate gradient
# #         dw1 = np.dot(x.T, delta1)
# #         # Gradient of biases equal to the error
# #         db2 = np.sum(delta2, axis=0, keepdims=True)
# #         db1 = np.sum(delta1, axis=0, keepdims=True)
# #         return dw2, db2, dw1, db1

In [6]:
# def cross_entropy_loss(y_pred, y):
#     return np.mean(-y * np.log(y_pred) - (1 - y) * np.log(1 - y_pred))

In [7]:
# def accuracy_score(y, y_pred):
#     y_pred_ = y_pred.argmax(axis=1)
#     y_ = y.argmax(axis=1)
#     return np.sum(y_pred_==y_) / y_.shape[0]

# from fault_tolerant_ml.ml.ops.tensor import Tensor

# t = Tensor(mnist.X_train)
# W = Tensor(np.random.randn(784, 128))
# b = Tensor(np.random.randn(1, 128))

# x1 = Tensor(np.array(3), requires_grad=True)
# x2 = Tensor(np.array(7), requires_grad=False)

# interim = ((x1 + x1) + x2)
# f = interim * interim

# $(2x_1 + x_2)^2 = 4x_1^2 + 4x_1x_2 +x_2^2$ 

# $\frac{\partial{df}}{\partial{x_1}} =8x_1 + 4x_2 = 24 + 28 = 52$

# $\frac{\partial{df}}{\partial{x_2}} =4x_1 + 2x_2 = 12 + 14 = 26$

# f.backward()

# x1.grad

# model = NeuralNet()
# model.add(nn.Layer(n_inputs=784, n_outputs=128))
# model.add(nn.Layer(n_inputs=128, n_outputs=10))
# y_pred = model.forward(mnist.X_train)

# l = 
# for layer in model.layers[::-1]:
#     print(layer)

In [8]:
# model = NeuralNet()
# print(model.fc1.shape)
# print(model.fc2.shape)
# epochs = 400
# learning_rate = 0.99
# m = mnist.X_train.shape[0]
# for epoch in np.arange(epochs):
    
#     # Feedforward
#     y_pred, z2, a1, z1 = model.forward(mnist.X_train)
    
#     # Calculate cost
#     loss = cross_entropy_loss(y_pred, mnist.y_train)
    
#     # Backprop
#     dw2, db2, dw1, db1 = model.backward(mnist.X_train, mnist.y_train, [y_pred, z2, a1, z1])
    
#     # Update weights
#     model.fc2.W = model.fc2.W - learning_rate * 1 / m * dw2
#     model.fc1.W = model.fc1.W - learning_rate * 1 / m * dw1
#     model.fc2.b = model.fc2.b - learning_rate * 1 / m * db2
#     model.fc1.b = model.fc1.b - learning_rate * 1 / m * db1
    
#     acc = accuracy_score(mnist.y_train, y_pred)
#     if epoch % 10 == 0:
#         print(f'epoch = {epoch}, loss = {loss:.3f}, TRAIN ACC = {acc:.3f}')
#     epoch += 1
    

## Autograd

## Graph

In [9]:
# g = Graph()

# g.set_as_default()

# X = Tensor(mnist.X_train)

# X + W

# W = Variable(np.random.rand(784, 128))

# b = Variable(np.zeros(shape=(1, 128)))

# z = add(matmul(X, W), b)

# z.input_nodes[0].input_nodes[0]

# z.compute()

In [10]:
# from fault_tolerant_ml.ml.ops import tensor as ft

# g = ft.Graph()
# g.set_as_default()
# X = ft.Tensor(mnist.X_train)
# y = ft.Tensor(mnist.y_train)

# W = ft.Tensor(np.random.randn(784, 128))

# b = ft.Tensor(np.zeros((1, 784)))

# a = ft.matmul(X, W)
# # z = ft.add(W, b)

# g.operations

# def evalulate(f):
#     val = []
#     for i, op in enumerate(f.operations):
#         print(*op.input_nodes)
#         val.append(op.compute(*op.input_nodes))

# def traverse(f):
    
#     operations = []
#     def recurse(node):
#         if isinstance(node, ft.Operation):
#             for input_node in node.input_nodes:
#                 recurse(input_node)
#         operations.append(node)
            
#     recurse(f)
#     return operations

# g = ft.Graph()
# g.set_as_default()
# x1 = ft.Tensor(np.array(3))
# x2 = ft.Tensor(np.array(7))

# f = ft.square(ft.add(ft.add(x1, x1), x2))

# traverse(f)

# g.operations[0].input_nodes

In [11]:
# evalulate(g)

In [12]:
# l = [ ("z1", "add", ("x1","x1")),
# ("z2", "add", ("z1","x2")),
# ("f", "square", ("z2",)) ]

# G = { "add" : lambda a,b: a+b,
# "square": lambda a:a*a }

In [13]:
# val = { "x1" : 3, "x2" : 7 }

# for step in l:
#     print(val)
#     var, op_name, func = step
#     lookup = list(map(val.get, func))
#     val[var] = G[op_name](*lookup)

In [14]:
# DG = { "add" : [ (lambda a,b: 1), (lambda a,b: 1) ],
# "square": [ lambda a:2*a ] }

# delta={}
# delta["f"] = 1
# for step in l[::-1]:
#     var, op_name, func = step
#     for op in DG[op_name]:
#         if var not in delta:
#             delta[var] = 0
#         lookup = list(map(val.get, func))
#         print(lookup)
#         delta[var] = delta[var] + DG[op_name](*lookup)

In [18]:
# class Tensor(object):
    
#     def __init__(self, data: np.ndarray, depends_on=None):
        
#         self.depends_on = depends_on or []
#         self.data = data
            
#     def __add__(self, other):
#         return Tensor(self.data + other.data, depends_on=[self, other])
    
#     def __pow__(self, p):
#         data = self.data ** p 
#         return Tensor(data, depends_on=[self])
    
#     def __repr__(self):
#         return f"Tensor({self.data}, dtype={self.data.dtype})"


# y1 = Tensor(np.array(3))
# y2 = Tensor(np.array(7))
# z1 = y1 + y1
# z2 = z1 ** 2

# z2.depends_on

# layers = []

In [19]:
from fault_tolerant_ml.activations.activation import Sigmoid

In [20]:
from typing import Union

In [1531]:
class Lay():
    
    def __init__(self, n_inputs, n_outputs):
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        self.W = Tensor((np.random.randn(self.n_inputs, self.n_outputs) * 0.01).astype(np.float32), is_param=True)
        self.b = Tensor(np.zeros((1, self.n_outputs)).astype(np.float32), is_param=True)
        
        self.activation_fn = Sigmoid()
        self.derived_vars = []
        
    def __call__(self, value):
        # Store input tensor for feedforward
        self.x = value
        # Store edge
        self.z = (value @ self.W) + self.b
        # Store output tensor for feedforward
        self.y = Tensor(self.activation_fn(self.z.data))
        
        return self.y
    
    def __repr__(self):
        return f"Layer({self.n_inputs}, {self.n_outputs})"

In [33]:
from fault_tolerant_ml.losses.loss_fns import CrossEntropyLoss
from fault_tolerant_ml.models import Model
from fault_tolerant_ml.layers import Layer
from fault_tolerant_ml.optimizers import SGD
from fault_tolerant_ml.metrics import accuracy_scorev2
import time

In [34]:
np.random.seed(42)

In [35]:
# Define loss
loss = CrossEntropyLoss()

# Define optimizer
optimizer = SGD(loss, learning_rate=0.99)

# Define model
model = Model(optimizer=optimizer)
l1 = Layer(784, 128)
# l2 = Lay(128, 128)
l3 = Layer(128, 10)
# Add layers
model.add([l1, l3])

# Tensorize numpy arrays
X_train = mnist.X_train
y_train = mnist.y_train
X_test = mnist.X_test
y_test = mnist.y_test
batch_size = 32

In [36]:
%%time
epochs = 20
for epoch in np.arange(epochs):
    
    epoch_loss = 0.0
    n_batches = 0
    for start in range(0, X_train.shape[0], batch_size):
        end = start + batch_size
    
        X_batch = X_train[start:end]
        y_batch = y_train[start:end]
        
        # Feedforward
        y_pred = model.forward(X_batch)

        # Calculate loss
        batch_loss = loss.loss(y_batch, y_pred, reduce=True).data

        # Backprop
        optimizer.compute_gradients(model, y_batch, y_pred)

        # Update gradients
        optimizer.apply_gradients(model)
        
        epoch_loss = epoch_loss + batch_loss
        n_batches += 1
        
    epoch_loss = epoch_loss / n_batches

    # Print norm of layer 0 weights
    print(f"Norm={np.linalg.norm(model.layers[0].W.data)}")
    # Calculate accuracy
    y_pred_train = model.forward(X_train)
    train_acc = accuracy_scorev2(y_train.data, y_pred_train.data)
    # Test accuracy
    y_pred_test = model.forward(X_test)
    test_acc = accuracy_scorev2(y_test.data, y_pred_test.data)

    if epoch % 1 == 0:
        print(f"Iteration {epoch}: Loss={epoch_loss:.4f}, train accuracy={train_acc:.4f}, test accuracy={test_acc:.4f}")
    epoch += 1

Norm=12.914712905883789
Iteration 0: Loss=0.1735, train accuracy=0.8987, test accuracy=0.9045
Norm=16.46483039855957
Iteration 1: Loss=0.1207, train accuracy=0.9244, test accuracy=0.9265
Norm=18.931461334228516
Iteration 2: Loss=0.1152, train accuracy=0.9393, test accuracy=0.9388
Norm=20.870220184326172
Iteration 3: Loss=0.1110, train accuracy=0.9487, test accuracy=0.9465
Norm=22.47098731994629
Iteration 4: Loss=0.1076, train accuracy=0.9548, test accuracy=0.9535
Norm=23.839521408081055
Iteration 5: Loss=0.1048, train accuracy=0.9596, test accuracy=0.9573
Norm=25.042911529541016
Iteration 6: Loss=0.1026, train accuracy=0.9637, test accuracy=0.9610
Norm=26.12214469909668
Iteration 7: Loss=0.1007, train accuracy=0.9672, test accuracy=0.9639
Norm=27.10328483581543
Iteration 8: Loss=0.0991, train accuracy=0.9701, test accuracy=0.9660
Norm=28.00339698791504
Iteration 9: Loss=0.0978, train accuracy=0.9723, test accuracy=0.9674
Norm=28.835487365722656
Iteration 10: Loss=0.0967, train accuracy

## Aggregate parameters for each layer

In [5]:
dWbs = []

n_workers = 7
n_layers = 2

In [6]:
l = []
l.append([1, 2])
l.append([2, 3])

In [7]:
l[1]

[2, 3]

In [8]:
worker_params = \
np.array([[[np.random.randn(784, 128), np.ones((1, 128))], [np.random.randn(128, 10), np.ones((1, 10))]] for worker in np.arange(n_workers)])

In [9]:
for i, worker_param in enumerate(worker_params):
    for j in np.arange(n_layers):
        print("worker ", i, "W=", worker_param[j][0].shape, end=", ")
        print("worker ", i,  "b=", worker_param[j][1].shape)

worker  0 W= (784, 128), worker  0 b= (1, 128)
worker  0 W= (128, 10), worker  0 b= (1, 10)
worker  1 W= (784, 128), worker  1 b= (1, 128)
worker  1 W= (128, 10), worker  1 b= (1, 10)
worker  2 W= (784, 128), worker  2 b= (1, 128)
worker  2 W= (128, 10), worker  2 b= (1, 10)
worker  3 W= (784, 128), worker  3 b= (1, 128)
worker  3 W= (128, 10), worker  3 b= (1, 10)
worker  4 W= (784, 128), worker  4 b= (1, 128)
worker  4 W= (128, 10), worker  4 b= (1, 10)
worker  5 W= (784, 128), worker  5 b= (1, 128)
worker  5 W= (128, 10), worker  5 b= (1, 10)
worker  6 W= (784, 128), worker  6 b= (1, 128)
worker  6 W= (128, 10), worker  6 b= (1, 10)


In [10]:
m = np.random.randint(-100, 100, size=(3, 3))

In [11]:
m

array([[ 97, -89, -73],
       [-49, -38, -64],
       [-87,  45,  89]])

In [12]:
(abs(m) / np.linalg.norm(m,1))

array([[0.41630901, 0.38197425, 0.31330472],
       [0.21030043, 0.16309013, 0.27467811],
       [0.37339056, 0.19313305, 0.38197425]])

In [13]:
m_ = m * 2

In [14]:
np.linalg.norm(m_)

438.7026327707642

In [15]:
m_ = m_ * np.exp(1 / 7)**-1

In [16]:
np.linalg.norm(m_)

380.3016169111953

In [17]:
np.linalg.norm(m * np.exp(8562 / 60000))

252.9962404903687

In [18]:
(8572 / 60000)

0.14286666666666667

In [19]:
np.linalg.norm(np.random.randn(10, 10) * (8572 / 60000))

1.4788280246697387

In [20]:
c = 1
n_sample_workers = int(np.ceil(7 * c))
np.random.choice(np.arange(7), n_sample_workers)

array([1, 6, 4, 3, 0, 6, 5])

In [23]:
errors = np.array([0.17, 0.18, 0.14, 0.20, 0.15])

In [24]:
errors

array([0.17, 0.18, 0.14, 0.2 , 0.15])

In [28]:
np.exp(errors) / np.sum(np.exp(errors))

array([0.20035467, 0.20236827, 0.1944333 , 0.20645638, 0.19638738])